In [1]:
import numpy as np
import pandas as pd
import scipy
import os
import sys
import matlab.engine

In [2]:
def Element2AtomNum(element = None, atomnum = None):
    Periodic_Table = {"H" :   1, "He" :   2, "Li" :   3, "Be" :   4, "B"  :   5, "C"  :   6, "N"  :   7, "O"  :   8, "F"  :   9, "Ne" :  10,
                     "Na" :  11, "Mg" :  12, "Al" :  13, "Si" :  14, "P"  :  15, "S"  :  16, "Cl" :  17, "Ar" :  18, "K"  :  19, "Ca" :  20,
                     "Sc" :  21, "Ti" :  22, "V"  :  23, "Cr" :  24, "Mn" :  25, "Fe" :  26, "Co" :  27, "Ni" :  28, "Cu" :  29, "Zn" :  30,
                     "Ga" :  31, "Ge" :  32, "As" :  33, "Se" :  34, "Br" :  35, "Kr" :  36, "Rb" :  37, "Sr" :  38, "Y"  :  39, "Zr" :  40,
                     "Nb" :  41, "Mo" :  42, "Tc" :  43, "Ru" :  44, "Rh" :  45, "Pd" :  46, "Ag" :  47, "Cd" :  48, "In" :  49, "Sn" :  50,
                     "Sb" :  51, "Te" :  52, "I"  :  53, "Xe" :  54, "Cs" :  55, "Ba" :  56, "La" :  57, "Ce" :  58, "Pr" :  59, "Nd" :  60,
                     "Pm" :  61, "Sm" :  62, "Eu" :  63, "Gd" :  64, "Tb" :  65, "Dy" :  66, "Ho" :  67, "Er" :  68, "Tm" :  69, "Yb" :  70,
                     "Lu" :  71, "Hf" :  72, "Ta" :  73, "W"  :  74, "Re" :  75, "Os" :  76, "Ir" :  77, "Pt" :  78, "Au" :  79, "Hg" :  80,
                     "Tl" :  81, "Pb" :  82, "Bi" :  83, "Po" :  84, "At" :  85, "Rn" :  86, "Fe" :  87, "Ra" :  88, "Ac" :  89, "Th" :  90,
                     "Pa" :  91, "U"  :  92, "Np" :  93, "Pu" :  94, "Am" :  95, "Cm" :  96, "Bk" :  97, "Cf" :  98, "Es" :  99, "Fm" : 100,
                     "Md" : 101, "No" : 102, "Lr" : 103, "Rf" : 104, "Db" : 105, "Sg" : 106, "Bh" : 107, "Hs" : 108, "Mt" : 109, "Ds" : 110,
                     "Rg" : 111, "Cn" : 112, "Uut": 113, "Fl" : 114, "Uup": 115, "Lv" : 116, "Uus": 117, "Uuo": 118}
    if element != None and atomnum == None:
        return Periodic_Table[element]
    if element == None and atomnum != None:
        return list(Periodic_Table.keys())[list(Periodic_Table.values()).index(atomnum)]
    if element == None and atomnum == None:
        return "Input Error: No Input"
    if element != None and atomnum != None:
        return "Input Error: Element2AtomNum() takes exactly one argument"

def Get_gjf(fname):
    nn = 0
    Atom_labels_list = []
    path=os.getcwd()
    fn = os.path.basename(fname)
    ifile = open( fname, 'r' )
    textlines = ifile.readlines()
    ifile.close()
    
    Coordinates = []
    for line in textlines[:] :
        if line == "\n":
            nn +=1
        if nn<1 :
            pass
        elif nn<2 :
            words = line.split()
            if len(words) > 0:
                Atom_labels_list += [int(x) for x in words[1:]]
        elif nn<3 :
            words = line.split()
            if len(words) > 2:
                atom_num = Element2AtomNum(element = words[0], atomnum = None)
                Coordinates += [[atom_num,float(words[1]),float(words[2]),float(words[3])]]
        else:
            break
    
    if len(Atom_labels_list) == 0:
        Atom_labels_list = [1]

    return Coordinates,Atom_labels_list

In [3]:
os.chdir(r' ')  #write path of BV.m
eng = matlab.engine.start_matlab()
eng.clc
eng.clear

In [4]:
def BV_matlab(AtomCoordinates,Atom_label):
    AtomCoordinateM = matlab.double(AtomCoordinates)
    future = eng.BV(AtomCoordinateM,Atom_label,nargout=3,async=True)
     ret_BV_3A,ret_BV_4A,ret_BV_5A = future.result()
    BV = [ret_BV_3A, ret_BV_4A, ret_BV_5A]
    return BV


In [5]:
def Get_BV(path):
    os.chdir(path)
    BV_M = []    
    
    from glob import glob
    n=0
    
    for eachfile in (glob('*.gjf') + glob('*.com')) :
        if (eachfile.split('-')[-1][-6:] == 'sp.com') or (eachfile.split('-')[-1][-6:] == 'sp.gjf'):
            n+=1
            print(eachfile)
            Coordinates,Atom_labels_list = Get_gjf(eachfile)
            for Atom_label in Atom_labels_list:
                BV = BV_matlab(Coordinates,Atom_label)
                print(BV)
                fn_ls = eachfile.split('-')
                fn = '-'.join([str(Atom_label),fn_ls[0]])   ##may need to be modified according to file name
                #fn = '-'.join([fn_ls[1],str(Atom_label),fn_ls[2],fn_ls[3]])  
                #fn = '-'.join([fn_ls[0],fn_ls[1],fn_ls[2],fn_ls[3],fn_ls[4],fn_ls[5],fn_ls[6]])
                BV_M += [[fn] + BV]
        else:
            pass
    print(n)
    Output = np.array(BV_M)
    return Output
    print('-------------------------------------------------------')

In [8]:
rootpath = r' '  ##root of the files with coordinates
os.chdir(rootpath)
Ar2Fixed_dir_list = os.listdir(rootpath)

BV_columns = ['fn','BV_3A','BV_4A','BV_5A']
BV_table = np.empty([0,4])

BV_dir = r'%s'%rootpath 
BV_table = np.r_[BV_table,Get_BV(BV_dir)]
                        
BV_table_all = pd.DataFrame(BV_table,columns=BV_columns)

os.chdir(rootpath)
BV_table_all.to_csv('BV_table.csv')

BV_table_all
#eng.quit()

POST-S09-P1-c1-1Osp.com
[0.5310835999999999, 0.2871768, 0.17409919999999998]
POST-S09-P2-c10-1Osp.com
[0.42913119999999993, 0.2530602, 0.15798379999999998]
POST-S09-P3-c1-1Osp.com
[0.5721658, 0.3424214, 0.20130599999999998]
POST-S19-P1-c5-1Osp.com
[0.5418322, 0.32296779999999997, 0.17936259999999998]
POST-S19-P2-c7-1Osp.com
[0.36059019999999997, 0.20897580000000002, 0.1351442]
POST-S20-P1-c12-1Osp.com
[0.6838192, 0.392102, 0.25396219999999997]
POST-S26-P1-c1-1Osp.com
[0.4996179999999999, 0.2682354, 0.1648232]
POST-S26-P2-c5-1Osp.com
[0.5006269999999999, 0.28336140000000004, 0.17260199999999998]
PRE-S09-c2-1Osp.com
[0.5465274, 0.2950323999999999, 0.17814240000000003]
[0.5615737999999999, 0.2994088, 0.1795604]
[0.5561584, 0.29953820000000003, 0.17970380000000002]
PRE-S19-c5-1Osp.com
[0.5294382000000001, 0.3029264, 0.179406]
[0.534308, 0.3149704, 0.1820736]
PRE-S20-c11-1Osp.com
[0.6679276, 0.360365, 0.22221319999999997]
PRE-S26-c3-1Osp.com
[0.5178056000000001, 0.2764766000000001, 0.171041

,fn,BV_3A,BV_4A,BV_5A
0,POST-S09-1-P1,0.5310835999999999,0.2871768,0.17409919999999998
1,POST-S09-1-P2,0.42913119999999993,0.2530602,0.15798379999999998
2,POST-S09-1-P3,0.5721658,0.3424214,0.20130599999999998
3,POST-S19-1-P1,0.5418322,0.32296779999999997,0.17936259999999998
4,POST-S19-1-P2,0.36059019999999997,0.20897580000000002,0.1351442
5,POST-S20-1-P1,0.6838192,0.392102,0.25396219999999997
6,POST-S26-1-P1,0.4996179999999999,0.2682354,0.1648232
7,POST-S26-1-P2,0.5006269999999999,0.28336140000000004,0.17260199999999998
8,PRE-S09-1-c2,0.5465274,0.2950323999999999,0.17814240000000003
9,PRE-S09-2-c2,0.5615737999999999,0.2994088,0.1795604


In [9]:
eng.quit()